In [ ]:
mn1,mn2,mx1,mx2=55.655155181884766, 37.49541473388672, 55.753211975097656, 37.75920104980469

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
range_lat=np.arange(55.655155, 55.753211975097656+0.0028016226632254465, 0.0028016226632254465)
range_long=np.arange(37.49541473388672, 37.75920104980469+0.007536751883370536, 0.007536751883370536)

In [ ]:
!pip install  overpy

In [ ]:
import overpy
api = overpy.Overpass()

def getarea(lat, lon, latend, lonend):
    coord = f"{lat},{lon},{latend},{lonend}"
    #[out:json];
    string = f"""node({coord}); out;
    """

    data = api.query(string)
    return data.nodes

#https://wiki.openstreetmap.org/wiki/RU:%D0%9A%D0%B0%D0%BA_%D0%BE%D0%B1%D0%BE%D0%B7%D0%BD%D0%B0%D1%87%D0%B8%D1%82%D1%8C
# Пример вывода данных
# zone = getarea()
# for node in zone:
#     if 'amenity' in node.tags:
#         print(f"amen= {node.tags['amenity']}")
#     if 'shop' in node.tags:
#         print(f"shop= {node.tags['shop']}")
#     print(f"lon= {node.lon} lat={node.lat} ")

In [ ]:
shift_lat=range_lat[1]-range_lat[0]
shift_long=range_long[1]-range_long[0]

In [ ]:
import time

In [ ]:
area_res=getarea(mn1,mn2,mx1,mx2)

In [70]:
def newarea(area_res, lat, lon, latend, longend):
  res=[]
  for i in area_res:
    if (lat<=i.lat<=latend and lon<=i.lon<=longend) and 'amenity' in i.tags.keys():
      res.append(i)
  return res

In [71]:
b=newarea(area_res, range_lat[0], range_long[0], range_lat[1], range_long[1])

In [72]:
for i in b:
  print(i.tags)



{'amenity': 'atm', 'cash_in': 'yes', 'description': 'Банкомат ДО «Ленинский»', 'opening_hours': '24/7', 'operator': 'ЗАО "ЮниКредит Банк"'}
{'amenity': 'restaurant', 'contact:phone': '+7 495 4380000', 'name': 'Тимур', 'opening_hours': 'Mo-Su 11:00-05:00'}
{'amenity': 'clinic', 'contact:phone': '+7 495 6041010', 'contact:website': 'http://www.stomed.ru', 'description': 'Для обследования требуют паспортные данные, номер моб. телефона и согласие на обработку персональных данных. На вопрос "будут ли данные переданы третьим лицам" отвечают "именно для этого мы их и спрашиваем".', 'healthcare': 'centre', 'name': 'Столица', 'opening_hours': 'Mo-Fr 08:00-16:00'}
{'amenity': 'driving_school', 'contact:phone': '+7 495 9703012', 'contact:website': 'http://www.avtostrela.ru', 'education': 'courses', 'education_form:parttime': 'yes', 'education_level:primary': 'yes', 'education_profile:driving': 'yes', 'education_system:ru_standard': 'yes', 'name': 'Стрела'}
{'amenity': 'bicycle_rental', 'capacity'

In [ ]:
def OSMGetData(area_res, lat, lon, latend, longend, amensp = set(), shopsp = set(), buildsp = set(), leisusp = set(), craftsp = set()):
  amenity_specs = amensp
  buildings_specs = buildsp
  shop_specs = shopsp
  leisure_specs = leisusp
  res=[]
  for i in area_res:
    f=0
    if (lat<=i.lat<=latend and lon<=i.lon<=longend):
      if 'amenity' in i.tags.keys():
        if len(set([i.tags['amenity']]).intersection(amenity_specs))>0:
          res.append(i)
          f=1
      elif not f and 'building' in i.tags.keys():
        if len(buildings_specs.intersection(set([i.tags['building']])))>0:
          res.append(i)
          f=1
      elif not f and 'shop' in i.tags.keys():
        if len(shop_specs.intersection(set([i.tags['shop']])))>0:
          res.append(i)
          f=1
      elif not f and 'leisure' in i.tags.keys():
        if len(leisure_specs.intersection(set([i.tags['leisure']])))>0:
          res.append(i)
          f=1
      elif not f and 'craft' in i.tags.keys():
        if len(leisure_specs.intersection(set([i.tags['craft']])))>0:
          res.append(i)
          f=1
  return res

In [ ]:
# ФУД РИТЕЙЛ 
# Мясо
sh_prm = {'butcher'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, shop_sp = sh_prm)[0].tags
# Булочная
sh_prm = {'bakery'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, shop_sp = sh_prm)[0].tags
# Остальное
sh_prm = {'supermarket', 'convenience'}
bld_prm = {'store', 'retail'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, shop_sp = sh_prm)[0].tags

# РЕСТОРАНЫ
# Кафе/Рестораны
am_prm = {'cafe', 'restaurant','fast_food', 'food_court'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, am_prm)[0].tags
# Клубы/Бары
am_prm = {'bar', 'biergarten',  'pub', 'nightclub', 'stripclub'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, am_prm, sh_prm, bld_prm, le_prm)[0].tags

# МЕД УСЛУГИ
# Клиника
am_prm = {'clinic'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, am_prm)[0].tags
# Вет клиника
am_prm = {'veterinary'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, am_prm)[0].tags
# Стоматология
am_prm = {'dentist'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, am_prm)[0].tags

# МАГАЗИНЫ
# Товары для дома
sh_prm = {'houseware', 'cosmetics', 'chemist', 'stationery'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, shop_sp = sh_prm)[0].tags
# Аптека
sh_prm = {'pharmacy'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, shop_sp = sh_prm)[0].tags
# Цветочный
sh_prm = {'florist'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, shop_sp = sh_prm)[0].tags

# САЛОНЫ
# Парикмахерская
sh_prm = {'hairdresser'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, shop_sp = sh_prm)[0].tags
# Салон красоты
am_prm ={'sauna'}
sh_prm = {'beauty', 'hairdresser','massage'}
le_prm={'sauna','tanning_salon'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, am_prm, sh_prm, leisusp = le_prm)[0].tags

# УСЛУГИ + ТОРГОВЛЯ
# Ремонт
craft_prm = {'watchmaker', 'electronics_repair'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, craftsp = craft_prm)[0].tags
# Фотоателье
sh_prm = {'photo', 'photo_studio'}
craft_prm = {'photographer'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, shop_sp = sh_prm, craftsp = craft_prm)[0].tags
# Табачка
sh_prm = {'tobacco'}
OSMGetData(area_res,mn1,mn2,mx1,mx2, shop_sp = sh_prm)[0].tags

In [80]:
def foodretail(area_res, lat, lon, latend, longend):
  amenity_specs=set()
  buildings_specs={'store', 'retail'}
  shop_specs={'supermarket', 'department_store','general'}
  res=[]
  for i in area_res:
    f=0
    if (lat<=i.lat<=latend and lon<=i.lon<=longend):
      if 'amenity' in i.tags.keys():
        if len(set([i.tags['amenity']]).intersection(amenity_specs))>0:
          res.append(i)
          f=1
      elif not f and 'building' in i.tags.keys():
        if len(buildings_specs.intersection(set([i.tags['building']])))>0:
          res.append(i)
          f=1
      elif not f and 'shop' in i.tags.keys():
        if len(shop_specs.intersection(set([i.tags['shop']])))>0:
          res.append(i)
          f=1
  return res

In [82]:
foodretail(area_res,mn1,mn2,mx1,mx2)[0].tags

{'alt_name': 'Пятерочка',
 'brand': 'Пятёрочка',
 'brand:wikidata': 'Q1768969',
 'brand:wikipedia': 'ru:Пятёрочка',
 'contact:facebook': 'https://www.facebook.com/pyaterochka',
 'contact:website': 'https://5ka.ru',
 'name': 'Пятёрочка',
 'name:en': 'Pyatyorochka',
 'name:ru': 'Пятёрочка',
 'opening_hours': 'Mo-Su 07:00-23:00',
 'shop': 'supermarket'}

In [84]:
def SPA(area_res, lat, lon, latend, longend):
  amenity_specs={'sauna'}
  leisure_specs={'sauna','tanning_salon'}
  buildings_specs=set()
  shop_specs={'beauty', 'hairdresser','massage'}
  res=[]
  for i in area_res:
    f=0
    if (lat<=i.lat<=latend and lon<=i.lon<=longend):
      if 'amenity' in i.tags.keys():
        if len(set([i.tags['amenity']]).intersection(amenity_specs))>0:
          res.append(i)
          f=1
      elif not f and 'building' in i.tags.keys():
        if len(buildings_specs.intersection(set([i.tags['building']])))>0:
          res.append(i)
          f=1
      elif not f and 'shop' in i.tags.keys():
        if len(shop_specs.intersection(set([i.tags['shop']])))>0:
          res.append(i)
          f=1
      elif not f and 'leisure' in i.tags.keys():
        if len(leisure_specs.intersection(set([i.tags['leisure']])))>0:
          res.append(i)
          f=1
  return res

In [85]:
SPA(area_res,mn1,mn2,mx1,mx2)[0].tags

{'name': 'Белиссима',
 'name:ru': 'Лантэкс центр',
 'opening_hours': 'Mo-Fr 09:00-21:00; Sa-Su 10:00-20:00',
 'phone': '+7 499 1796449',
 'shop': 'hairdresser'}

In [86]:
#Рестораны, кафе, бары и ночные клубы
def cafes(area_res, lat, lon, latend, longend):
  amenity_specs={'bar', 'biergarten', 'cafe', 'restaurant', 'pub', 'nightclub', 'stripclub', 'fast_food', 'food_court'}
  leisure_specs=set()
  buildings_specs=set()
  shop_specs=set()
  res=[]
  for i in area_res:
    f=0
    if (lat<=i.lat<=latend and lon<=i.lon<=longend):
      if 'amenity' in i.tags.keys():
        if len(set([i.tags['amenity']]).intersection(amenity_specs))>0:
          res.append(i)
          f=1
      elif not f and 'building' in i.tags.keys():
        if len(buildings_specs.intersection(set([i.tags['building']])))>0:
          res.append(i)
          f=1
      elif not f and 'shop' in i.tags.keys():
        if len(shop_specs.intersection(set([i.tags['shop']])))>0:
          res.append(i)
          f=1
      elif not f and 'leisure' in i.tags.keys():
        if len(leisure_specs.intersection(set([i.tags['leisure']])))>0:
          res.append(i)
          f=1
  return res

In [87]:
cafes(area_res,mn1,mn2,mx1,mx2)[0].tags

{'alt_name': 'Kentucky Fried Chicken',
 'amenity': 'fast_food',
 'brand': 'KFC',
 'brand:wikidata': 'Q524757',
 'brand:wikipedia': 'en:KFC',
 'cuisine': 'chicken',
 'level': '3',
 'name': 'KFC',
 'takeaway': 'yes'}

In [88]:
#4. Бытовая химия, магазины косметики и т.п.
def chemist(area_res, lat, lon, latend, longend):
  amenity_specs=set()
  leisure_specs=set()
  buildings_specs=set()
  shop_specs={'chemist', 'cosmetics', 'hairdresser_supply', 'perfumery'}
  res=[]
  for i in area_res:
    f=0
    if (lat<=i.lat<=latend and lon<=i.lon<=longend):
      if 'amenity' in i.tags.keys():
        if len(set([i.tags['amenity']]).intersection(amenity_specs))>0:
          res.append(i)
          f=1
      elif not f and 'building' in i.tags.keys():
        if len(buildings_specs.intersection(set([i.tags['building']])))>0:
          res.append(i)
          f=1
      elif not f and 'shop' in i.tags.keys():
        if len(shop_specs.intersection(set([i.tags['shop']])))>0:
          res.append(i)
          f=1
      elif not f and 'leisure' in i.tags.keys():
        if len(leisure_specs.intersection(set([i.tags['leisure']])))>0:
          res.append(i)
          f=1
  return res

In [89]:
chemist(area_res,mn1,mn2,mx1,mx2)[0].tags

{'addr:housenumber': '4',
 'addr:street': 'Университетский проспект',
 'contact:website': 'http://www.ugdvor.ru',
 'name': 'Южный двор',
 'opening_hours': 'Mo-Su 09:00-22:00',
 'shop': 'chemist'}

In [90]:
#5. Клиники, стоматологии, вет.клиники
def meds(area_res, lat, lon, latend, longend):
  amenity_specs={'veterinary', 'clinic', 'dentist'}
  leisure_specs=set()
  buildings_specs=set()
  shop_specs=set()
  res=[]
  for i in area_res:
    f=0
    if (lat<=i.lat<=latend and lon<=i.lon<=longend):
      if 'amenity' in i.tags.keys():
        if len(set([i.tags['amenity']]).intersection(amenity_specs))>0:
          res.append(i)
          f=1
      elif not f and 'building' in i.tags.keys():
        if len(buildings_specs.intersection(set([i.tags['building']])))>0:
          res.append(i)
          f=1
      elif not f and 'shop' in i.tags.keys():
        if len(shop_specs.intersection(set([i.tags['shop']])))>0:
          res.append(i)
          f=1
      elif not f and 'leisure' in i.tags.keys():
        if len(leisure_specs.intersection(set([i.tags['leisure']])))>0:
          res.append(i)
          f=1
  return res

In [95]:
meds(area_res,mn1,mn2,mx1,mx2)[3].tags

{'addr:housenumber': '10Б',
 'addr:street': 'улица Академика Челомея',
 'amenity': 'clinic',
 'health_facility:type': 'clinic',
 'healthcare': 'clinic',
 'name': 'Прима Медика',
 'name:en': 'Prima Medica',
 'phone': '+7 495 2582559',
 'website': 'https://www.primamedica.ru/'}

In [98]:
#6. Услуги (кофе на вынос, ремонт обуви и ювелирных изделий) и мелкая торговля (фото, пресса, мороженое).
def small(area_res, lat, lon, latend, longend):
  amenity_specs=set()
  leisure_specs=set()
  buildings_specs={'kiosk'}
  shop_specs={'newsagent', 'convinience'}
  res=[]
  for i in area_res:
    f=0
    if (lat<=i.lat<=latend and lon<=i.lon<=longend):
      if 'amenity' in i.tags.keys():
        if len(set([i.tags['amenity']]).intersection(amenity_specs))>0:
          res.append(i)
          f=1
      elif not f and 'building' in i.tags.keys():
        if len(buildings_specs.intersection(set([i.tags['building']])))>0:
          res.append(i)
          f=1
      elif not f and 'building:levels' in i.tags.keys() and i.tags['building:levels']=='1':
        res.append(i)
        f=1
      elif not f and 'shop' in i.tags.keys():
        if len(shop_specs.intersection(set([i.tags['shop']])))>0:
          res.append(i)
          f=1
      elif not f and 'leisure' in i.tags.keys():
        if len(leisure_specs.intersection(set([i.tags['leisure']])))>0:
          res.append(i)
          f=1
  return res

In [99]:
small(area_res,mn1,mn2,mx1,mx2)[3].tags

{'building': 'kiosk', 'building:levels': '1', 'shop': 'newsagent'}